In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
def get_secret(secret_name='wysde'):
    import boto3
    import json
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name)
    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    get_secret_value_response = json.loads(get_secret_value_response['SecretString'])
    return get_secret_value_response

secret_vals = get_secret()

redshift_endpoint1 = secret_vals['REDSHIFT_HOST']
redshift_user1 = secret_vals['REDSHIFT_USERNAME']
redshift_pass1 = secret_vals['REDSHIFT_PASSWORD']
port1 = 5439
dbname1 = "dev"

In [3]:
engine_string = "postgresql+psycopg2://%s:%s@%s:%d/%s" \
% (redshift_user1, redshift_pass1, redshift_endpoint1, port1, dbname1)
engine = create_engine(engine_string)

In [4]:
query = """
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
"""

df = pd.read_sql_query(text(query), engine)
df

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers
0,pg_internal,redshift_auto_health_check_13983357,rdsdb,None,False,False,False
1,public,dwdate,admin,None,False,False,False
2,public,lineitem,admin,None,False,False,False
3,public,supplier,admin,None,False,False,False
4,public,part,admin,None,False,False,False
5,public,orders,admin,None,False,False,False
6,public,customer,admin,None,False,False,False


In [11]:
query = """
SELECT * FROM "dev"."public"."orders" limit 10;
"""

df = pd.read_sql_query(text(query), engine)
df

,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
